In [102]:
import pandas as pd
from datetime import datetime
from defs import print_with_time, raw_data_dir, interim_data_dir, depois_do_feriado
from tqdm import tqdm



ontem = pd.to_datetime(datetime.today().date()) - pd.Timedelta(1, 'd') + pd.Timedelta(19, 'h')
df0 = pd.read_pickle(raw_data_dir+'/query_result.pickle')

assert df0['dt_entrada_unidade'].max() >= ontem, \
    print(f"Erro: não há entradas registradas no dia de ontem ({ontem.strftime('%d/%m/%Y')}). Data máxima: {df0['dt_entrada_unidade'].max()}")
assert df0['dt_saida_unidade'].max() >= ontem, \
    print(f"Erro: não há saídas registradas no dia de ontem ({ontem.strftime('%d/%m/%Y')}). Data máxima: {df0['dt_saida_unidade'].max()}")

# Tirando pacientes eletivos
df_ = df0.reset_index().copy()
first = df_.sort_values('dt_entrada_unidade').groupby('nr_atendimento').first().reset_index()
first_eletivo_idxs = first[first['ds_carater_internacao'] == 'Eletivo']['index'].unique()
df_entrada = df_.loc[~df_['index'].isin(first_eletivo_idxs)].copy()

Erro: não há entradas registradas no dia de ontem (24/04/2022). Data máxima: 2022-04-19 23:58:07


AssertionError: None

In [ ]:
first_eletivo_idxs

In [ ]:
saida_ui = pd.read_pickle(interim_data_dir+'/saida_ui_horario.pickle')

In [ ]:
periodos_mapper = {
    6: '06-12',
    12: '12-18',
    18: '18-24',
    0: '00-06',
}

saida_ui['ds'].dt.hour.map(periodos_mapper)

In [113]:


daily_prediction_to_period_prediction()

In [116]:
preds = read_pickle(final_data_dir+'/previsoes.pickle')
new_preds = pd.DataFrame()
new_preds = new_preds.append([preds]*4).sort_values(['dt_previsao','tipo','ds_classific_setor']).reset_index(drop=True)

setores = ['ui', 'uti']
tipos = ['entrada', 'saida']
hrr_previsao_idx = list(new_preds.columns).index('hrr_previsao')
qtd_previsao_idx = list(new_preds.columns).index('qtd_previsao')
qtd_previsao_max_idx = list(new_preds.columns).index('qtd_previsao_max')
qtd_previsao_min_idx = list(new_preds.columns).index('qtd_previsao_min')
dias = new_preds['dt_previsao'].unique()

for setor in setores:
    for tipo in tipos:
        with open(interim_data_dir+f'/weekday_info_{tipo}_{setor}.json') as weekday_file:
            weekday_info = json.load(weekday_file)
        
        for dia in dias:
            df_ = new_preds[(new_preds['tipo'] == tipo.capitalize()) &
                            (new_preds['ds_classific_setor'] == setor.upper()) &
                            (new_preds['dt_previsao'] == dia)].copy()
            assert len(df_) == 4, "Print df_ maior que 4"
            qtd_previsao = df_['qtd_previsao'].iloc[0]
            qtd_previsao_max = df_['qtd_previsao_max'].iloc[0]
            qtd_previsao_min = df_['qtd_previsao_min'].iloc[0]
            weekday = df_.iloc[0]['dt_previsao'].weekday()
            this_weekday_info = weekday_info[str(weekday)]
            for i, periodo in enumerate(this_weekday_info.keys()):
                df_.iloc[i, hrr_previsao_idx] = periodo
                df_.iloc[i, qtd_previsao_idx] = qtd_previsao * this_weekday_info[periodo]
                df_.iloc[i, qtd_previsao_max_idx] = qtd_previsao_max * this_weekday_info[periodo]
                df_.iloc[i, qtd_previsao_min_idx] = qtd_previsao_min * this_weekday_info[periodo]
                
            new_preds.loc[df_.index] = df_

{'00-06': 0.1206772704, '06-12': 0.1297075423, '12-18': 0.4135454079, '18-24': 0.3360697794}
{'00-06': 0.1234489341, '06-12': 0.1231307668, '12-18': 0.4064057694, '18-24': 0.3470145296}
{'00-06': 0.1232444203, '06-12': 0.1243331519, '12-18': 0.4047904192, '18-24': 0.3476320087}
{'00-06': 0.1488625782, '06-12': 0.1207928695, '12-18': 0.4171877079, '18-24': 0.3131568445}
{'00-06': 0.1294864716, '06-12': 0.1187189398, '12-18': 0.4250414136, '18-24': 0.326753175}
{'00-06': 0.0937010955, '06-12': 0.1550273865, '12-18': 0.4285015649, '18-24': 0.3227699531}
{'00-06': 0.1315042573, '06-12': 0.1416272469, '12-18': 0.4092715232, '18-24': 0.3175969726}
{'00-06': 0.0299983334, '06-12': 0.2857619021, '12-18': 0.484250875, '18-24': 0.1999888895}
{'00-06': 0.0303687636, '06-12': 0.2843275488, '12-18': 0.4877440347, '18-24': 0.1975596529}
{'00-06': 0.0259590024, '06-12': 0.2672250243, '12-18': 0.4962287724, '18-24': 0.210587201}
{'00-06': 0.0344923815, '06-12': 0.3083105431, '12-18': 0.506892895, '18-

In [115]:
print((preds['qtd_previsao'].sum() - new_preds['qtd_previsao'].sum()) == 0,
      (preds['qtd_previsao_max'].sum() - new_preds['qtd_previsao_max'].sum()) == 0,
      (preds['qtd_previsao_min'].sum() - new_preds['qtd_previsao_min'].sum()) == 0)

False False False


In [126]:
print((preds['qtd_previsao'].sum() - new_preds['qtd_previsao'].sum())*10e6,
      (preds['qtd_previsao_max'].sum() - new_preds['qtd_previsao_max'].sum())*10e6,
      (preds['qtd_previsao_min'].sum() - new_preds['qtd_previsao_min'].sum())*10e6)

-0.03500076672935393 -0.03899913281202316 -0.03100012690993026


In [7]:
import pandas as pd
from datetime import datetime
datetime.today().date() - pd.Timedelta(1, 'd')

datetime.date(2022, 4, 25)

In [13]:
import pandas as pd
df = pd.read_pickle('/home/ffreller/EntradaESaida/V4_Periodos/data/final/previsoes_periodos.pickle')
df.dt_previsao.min()

Timestamp('2022-04-27 00:00:00')